# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [7]:
# Load the https://www.residentadvisor.net/events page in your browser.
import requests
from bs4 import BeautifulSoup

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [2]:
# Open the inspect element feature in your browser
response = requests.get("https://www.residentadvisor.net/events/us/newyork")
soup = BeautifulSoup(response.content, 'html.parser')

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [3]:
event_listings = soup.find('div', id="event-listing")

In [4]:
entries = event_listings.findAll('li')
print(len(entries), entries[0])

13 <li><p class="eventDate date"><a href="/events.aspx?ai=8&amp;v=day&amp;mn=7&amp;yr=2020&amp;dy=3"><span>Fri, 03 Jul 2020 /</span></a></p></li>


In [5]:
import pandas as pd
import numpy as np
import re
import time

In [6]:
#Successive exploration in function development
rows = []
for entry in entries:
    #Is it a date? If so, set current date.
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()

,0,1,2,3
0,Rona ITW Music Festival,North Jersey Forest,"Fri, 03 Jul 2020 /",4.0
1,NYC Independence Day Weekend Yacht Party Cruis...,Skyport Marina,"Fri, 03 Jul 2020 /",1.0
2,NYC July 4th Weekend Hip Hop vs Reggae® Yacht ...,Skyport Marina,"Fri, 03 Jul 2020 /",NaN
3,Rona ITW Music Festival,North Jersey Forest,"Sat, 04 Jul 2020 /",4.0
4,4th of July Independance Fireworks Cruise,Harmony Yacht,"Sat, 04 Jul 2020 /",1.0


In [9]:
def scrape_events(events_page_url):
    #Your code here
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

In [10]:
df

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Rona ITW Music Festival,North Jersey Forest,"Fri, 03 Jul 2020 /",4.0
1,NYC Independence Day Weekend Yacht Party Cruis...,Skyport Marina,"Fri, 03 Jul 2020 /",1.0
2,NYC July 4th Weekend Hip Hop vs Reggae® Yacht ...,Skyport Marina,"Fri, 03 Jul 2020 /",NaN
3,Rona ITW Music Festival,North Jersey Forest,"Sat, 04 Jul 2020 /",4.0
4,4th of July Independance Fireworks Cruise,Harmony Yacht,"Sat, 04 Jul 2020 /",1.0
5,4th of July Midnight Yacht Cruise,Harbor Lights Yacht,"Sat, 04 Jul 2020 /",NaN
6,4th of July Fireworks Cruise,Cosmo,"Sat, 04 Jul 2020 /",NaN
7,4th of July Independance Fireworks Cruise,Harmony Yacht,"Sun, 05 Jul 2020 /",1.0
8,4th of July Fireworks Cruise,Cosmo,"Sun, 05 Jul 2020 /",NaN


## Write a Function to Retrieve the URL for the Next Page

In [12]:
def next_page(url):
    #Your code here
    soup = BeautifulSoup(response.content, 'lxml')
    url_ext = soup.find('a',attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [13]:
#Your code here
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/newyork"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

1000


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Rona ITW Music Festival,North Jersey Forest,"Fri, 03 Jul 2020 /",4.0
1,NYC Independence Day Weekend Yacht Party Cruis...,Skyport Marina,"Fri, 03 Jul 2020 /",1.0
2,NYC July 4th Weekend Hip Hop vs Reggae® Yacht ...,Skyport Marina,"Fri, 03 Jul 2020 /",NaN
3,Rona ITW Music Festival,North Jersey Forest,"Sat, 04 Jul 2020 /",4.0
4,4th of July Independance Fireworks Cruise,Harmony Yacht,"Sat, 04 Jul 2020 /",1.0


## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!